In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
from scipy.stats import zscore
import scipy.stats as st
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [4]:
df = pd.read_csv('HI-Small_Trans.csv')
df

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.340000,US Dollar,3697.340000,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.010000,US Dollar,0.010000,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.570000,US Dollar,14675.570000,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.970000,US Dollar,2806.970000,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.970000,US Dollar,36682.970000,US Dollar,Reinvestment,0
...,...,...,...,...,...,...,...,...,...,...,...
5078340,2022/09/10 23:57,54219,8148A6631,256398,8148A8711,0.154978,Bitcoin,0.154978,Bitcoin,Bitcoin,0
5078341,2022/09/10 23:35,15,8148A8671,256398,8148A8711,0.108128,Bitcoin,0.108128,Bitcoin,Bitcoin,0
5078342,2022/09/10 23:52,154365,8148A6771,256398,8148A8711,0.004988,Bitcoin,0.004988,Bitcoin,Bitcoin,0
5078343,2022/09/10 23:46,256398,8148A6311,256398,8148A8711,0.038417,Bitcoin,0.038417,Bitcoin,Bitcoin,0


In [5]:
missing_data = df.isnull().sum()
missing_data

,0
Timestamp,0
From Bank,0
Account,0
To Bank,0
Account.1,0
Amount Received,0
Receiving Currency,0
Amount Paid,0
Payment Currency,0
Payment Format,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078345 entries, 0 to 5078344
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           object 
 1   From Bank           int64  
 2   Account             object 
 3   To Bank             int64  
 4   Account.1           object 
 5   Amount Received     float64
 6   Receiving Currency  object 
 7   Amount Paid         float64
 8   Payment Currency    object 
 9   Payment Format      object 
 10  Is Laundering       int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 426.2+ MB


In [7]:
#Checking for duplicates
num_duplicates = df.duplicated(keep = False).sum()
print(f"Number of duplicate rows: {num_duplicates}")

Number of duplicate rows: 18


In [8]:
df.duplicated()

,0
0,False
1,False
2,False
3,False
4,False
...,...
5078340,False
5078341,False
5078342,False
5078343,False


In [9]:
duplicated_rows = df[df.duplicated(keep = False)]
print("Duplicated rows:")
duplicated_rows.sort_values(by = ['Account.1','Account'])

Duplicated rows:


,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
863293,2022/09/01 16:20,12004,800C927C1,12004,800C927C0,0.000008,Bitcoin,0.080000,Euro,ACH,0
863295,2022/09/01 16:20,12004,800C927C1,12004,800C927C0,0.000008,Bitcoin,0.080000,Euro,ACH,0
4816510,2022/09/09 21:33,14433,80935A891,14433,80935A890,0.000001,Bitcoin,0.010000,US Dollar,ACH,0
4816512,2022/09/09 21:33,14433,80935A891,14433,80935A890,0.000001,Bitcoin,0.010000,US Dollar,ACH,0
4510478,2022/09/09 10:03,6075,80C702911,6075,80C702910,0.000002,Bitcoin,0.020000,US Dollar,ACH,0
4510480,2022/09/09 10:03,6075,80C702911,6075,80C702910,0.000002,Bitcoin,0.020000,US Dollar,ACH,0
4166784,2022/09/08 21:05,113779,811144AB1,113779,811144AB0,0.000002,Bitcoin,0.020000,US Dollar,ACH,0
4166786,2022/09/08 21:05,113779,811144AB1,113779,811144AB0,0.000002,Bitcoin,0.020000,US Dollar,ACH,0
3684003,2022/09/07 21:25,29992,8099A29B1,220,813725AE1,0.000003,Bitcoin,0.000003,Bitcoin,Bitcoin,0
3684005,2022/09/07 21:25,29992,8099A29B1,220,813725AE1,0.000003,Bitcoin,0.000003,Bitcoin,Bitcoin,0


In [10]:
df = df.drop_duplicates()

In [11]:
df.describe()

,From Bank,To Bank,Amount Received,Amount Paid,Is Laundering
count,5.078336e+06,5.078336e+06,5.078336e+06,5.078336e+06,5.078336e+06
mean,4.573058e+04,6.574461e+04,5.988737e+06,4.509281e+06,1.019428e-03
std,8.176567e+04,8.409303e+04,1.037184e+09,8.697736e+08,3.191221e-02
min,1.000000e+00,1.000000e+00,1.000000e-06,1.000000e-06,0.000000e+00
25%,1.190000e+02,4.259000e+03,1.833700e+02,1.844800e+02,0.000000e+00
50%,9.679000e+03,2.156800e+04,1.411040e+03,1.414570e+03,0.000000e+00
75%,2.862800e+04,1.223320e+05,1.234630e+04,1.229815e+04,0.000000e+00
max,3.563030e+05,3.562940e+05,1.046302e+12,1.046302e+12,1.000000e+00


In [12]:
df = df.rename(columns = {'Account': 'Recieving_account', 'Account.1':'Originating_Account'})

In [ ]:
df.isnull().any()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5078336 entries, 0 to 5078344
Data columns (total 11 columns):
 #   Column               Dtype  
---  ------               -----  
 0   Timestamp            object 
 1   From Bank            int64  
 2   Recieving_account    object 
 3   To Bank              int64  
 4   Originating_Account  object 
 5   Amount Received      float64
 6   Receiving Currency   object 
 7   Amount Paid          float64
 8   Payment Currency     object 
 9   Payment Format       object 
 10  Is Laundering        int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 464.9+ MB


Based on the EDA for this dataset, these are the features that we found that these are the following features that turned out to be good predictors for whether a particular transaction is laundering or not:



* The time stamp of a transaction
* The originating account
* The recieving account
* The amount recieved
* The amount paid
* The payment currency
* The Recieving currency
* The format of the payment

So let us work with these features and try to feature engineer them. Let us try to drop all the other features.

In [ ]:
df = df.drop(columns=['From Bank', 'To Bank'])

In [ ]:
df.head()

,Timestamp,Recieving_account,Originating_Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,8000EBD30,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,8000F4580,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,8000F4670,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,8000F5030,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,8000F5200,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


### Feature Engineering the Time Stamp Column

In [ ]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y/%m/%d %H:%M')


Getting the `Year`, `Month`, and `Day`.

In [ ]:
df['Year'] = df['Timestamp'].dt.year
df['Month'] = df['Timestamp'].dt.month
df['Day'] = df['Timestamp'].dt.day

Getting the `Hour` and `Minute`.

In [ ]:
df['Hour'] = df['Timestamp'].dt.hour
df['Minute'] = df['Timestamp'].dt.minute

Getting `Day_of_week`.

In [ ]:
df['Day_of_Week'] = df['Timestamp'].dt.dayofweek

Getting `Is_Weekend`.

In [ ]:
df['Is_Weekend'] = df['Day_of_Week'].apply(lambda x: 1 if x >= 5 else 0)

In [ ]:
def get_part_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['Part_of_Day'] = df['Hour'].apply(get_part_of_day)

In [ ]:
df = df.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute', 'Day_of_Week', 'Is_Weekend', 'Part_of_Day'])


In [ ]:
df

,Timestamp,Recieving_account,Originating_Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Year_Norm,Month_Norm,Day_Norm,Hour_Norm,Minute_Norm,Day_of_Week_Norm
0,2022-09-01 00:20:00,8000EBD30,8000EBD30,3697.340000,US Dollar,3697.340000,US Dollar,Reinvestment,0,NaN,0.727273,0.0,0.0,0.338983,0.500000
1,2022-09-01 00:20:00,8000F4580,8000F5340,0.010000,US Dollar,0.010000,US Dollar,Cheque,0,NaN,0.727273,0.0,0.0,0.338983,0.500000
2,2022-09-01 00:00:00,8000F4670,8000F4670,14675.570000,US Dollar,14675.570000,US Dollar,Reinvestment,0,NaN,0.727273,0.0,0.0,0.000000,0.500000
3,2022-09-01 00:02:00,8000F5030,8000F5030,2806.970000,US Dollar,2806.970000,US Dollar,Reinvestment,0,NaN,0.727273,0.0,0.0,0.033898,0.500000
4,2022-09-01 00:06:00,8000F5200,8000F5200,36682.970000,US Dollar,36682.970000,US Dollar,Reinvestment,0,NaN,0.727273,0.0,0.0,0.101695,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5078340,2022-09-10 23:57:00,8148A6631,8148A8711,0.154978,Bitcoin,0.154978,Bitcoin,Bitcoin,0,NaN,0.727273,0.3,1.0,0.966102,0.833333
5078341,2022-09-10 23:35:00,8148A8671,8148A8711,0.108128,Bitcoin,0.108128,Bitcoin,Bitcoin,0,NaN,0.727273,0.3,1.0,0.593220,0.833333
5078342,2022-09-10 23:52:00,8148A6771,8148A8711,0.004988,Bitcoin,0.004988,Bitcoin,Bitcoin,0,NaN,0.727273,0.3,1.0,0.881356,0.833333
5078343,2022-09-10 23:46:00,8148A6311,8148A8711,0.038417,Bitcoin,0.038417,Bitcoin,Bitcoin,0,NaN,0.727273,0.3,1.0,0.779661,0.833333


I'm not going to standardize this data, because I plan to use classification models like random forest and decison trees for  predicting whether a transaction is fraudulent or not, and they don't need data to be normalized.

### Feature Engineering the **originating account**  and **recieving account**

There are really no new features we can make from the Originating and Recieving account. I'm not going to normalize this data because I'll be passing it into a classification algorithm.

In [ ]:
df['Originating_Account']

,Originating_Account
0,8000EBD30
1,8000F5340
2,8000F4670
3,8000F5030
4,8000F5200
...,...
5078340,8148A8711
5078341,8148A8711
5078342,8148A8711
5078343,8148A8711


In [ ]:
df['Recieving_account']

,Recieving_account
0,8000EBD30
1,8000F4580
2,8000F4670
3,8000F5030
4,8000F5200
...,...
5078340,8148A6631
5078341,8148A8671
5078342,8148A6771
5078343,8148A6311


In [ ]:
df['Account Match'] = (df['Originating_Account'] == df['Recieving_account']).astype(int)
df['Account Match'].value_counts()

,count
Account Match,
0,4487124
1,591212


## Feature Engineering the **amount received** and **amount paid**

I am not going to standardize this data because I'm going to be using the random forest and decision tree algorithm to make predictions.
I'm going to create a new feature called `Amount Paid and Recieved Match` in order to extract how many of the amounts matched and did not.

In [ ]:
df['Amount Received']

,Amount Received
0,3697.340000
1,0.010000
2,14675.570000
3,2806.970000
4,36682.970000
...,...
5078340,0.154978
5078341,0.108128
5078342,0.004988
5078343,0.038417


In [ ]:
df['Amount Paid']

,Amount Paid
0,3697.340000
1,0.010000
2,14675.570000
3,2806.970000
4,36682.970000
...,...
5078340,0.154978
5078341,0.108128
5078342,0.004988
5078343,0.038417


In [ ]:
df['Amount Paid and Recieved Match'] = (df['Amount Received'] == df['Amount Paid']).astype(int)
df['Amount Paid and Recieved Match']

,Amount Paid and Recieved Match
0,1
1,1
2,1
3,1
4,1
...,...
5078340,1
5078341,1
5078342,1
5078343,1


In [ ]:
df['Amount Paid and Recieved Match'].value_counts()

,count
Amount Paid and Recieved Match,
1,5006182
0,72154


## Feature engineering the currency of the payment and the recieving currency

In [ ]:
df['Payment Currency']

,Payment Currency
0,US Dollar
1,US Dollar
2,US Dollar
3,US Dollar
4,US Dollar
...,...
5078340,Bitcoin
5078341,Bitcoin
5078342,Bitcoin
5078343,Bitcoin


In [ ]:
df['Receiving Currency']

,Receiving Currency
0,US Dollar
1,US Dollar
2,US Dollar
3,US Dollar
4,US Dollar
...,...
5078340,Bitcoin
5078341,Bitcoin
5078342,Bitcoin
5078343,Bitcoin


In [ ]:
df['Currency Match'] = (df['Payment Currency'] == df['Receiving Currency']).astype(int)
df['Currency Match'].value_counts()

,count
Currency Match,
1,5006170
0,72166


## Feature engineering the payment format

In [ ]:
df['Payment Format']

,Payment Format
0,Reinvestment
1,Cheque
2,Reinvestment
3,Reinvestment
4,Reinvestment
...,...
5078340,Bitcoin
5078341,Bitcoin
5078342,Bitcoin
5078343,Bitcoin


In [ ]:
df

,Timestamp,Recieving_account,Originating_Account,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Year_Norm,Month_Norm,Day_Norm,Hour_Norm,Minute_Norm,Day_of_Week_Norm,Amount Paid and Recieved Match,Account Match,Currency Match
0,2022-09-01 00:20:00,8000EBD30,8000EBD30,3697.340000,US Dollar,3697.340000,US Dollar,Reinvestment,0,NaN,0.727273,0.0,0.0,0.338983,0.500000,1,1,1
1,2022-09-01 00:20:00,8000F4580,8000F5340,0.010000,US Dollar,0.010000,US Dollar,Cheque,0,NaN,0.727273,0.0,0.0,0.338983,0.500000,1,0,1
2,2022-09-01 00:00:00,8000F4670,8000F4670,14675.570000,US Dollar,14675.570000,US Dollar,Reinvestment,0,NaN,0.727273,0.0,0.0,0.000000,0.500000,1,1,1
3,2022-09-01 00:02:00,8000F5030,8000F5030,2806.970000,US Dollar,2806.970000,US Dollar,Reinvestment,0,NaN,0.727273,0.0,0.0,0.033898,0.500000,1,1,1
4,2022-09-01 00:06:00,8000F5200,8000F5200,36682.970000,US Dollar,36682.970000,US Dollar,Reinvestment,0,NaN,0.727273,0.0,0.0,0.101695,0.500000,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5078340,2022-09-10 23:57:00,8148A6631,8148A8711,0.154978,Bitcoin,0.154978,Bitcoin,Bitcoin,0,NaN,0.727273,0.3,1.0,0.966102,0.833333,1,0,1
5078341,2022-09-10 23:35:00,8148A8671,8148A8711,0.108128,Bitcoin,0.108128,Bitcoin,Bitcoin,0,NaN,0.727273,0.3,1.0,0.593220,0.833333,1,0,1
5078342,2022-09-10 23:52:00,8148A6771,8148A8711,0.004988,Bitcoin,0.004988,Bitcoin,Bitcoin,0,NaN,0.727273,0.3,1.0,0.881356,0.833333,1,0,1
5078343,2022-09-10 23:46:00,8148A6311,8148A8711,0.038417,Bitcoin,0.038417,Bitcoin,Bitcoin,0,NaN,0.727273,0.3,1.0,0.779661,0.833333,1,0,1


In [ ]:
df.to_csv('feature_engineered_dataset.csv', index=False)